# Recurrent Neural Networks RNN

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Drawbacks to non-sequential models</p>

In earlier notebooks, we saw how to use **Bag-of-Words** like approaches to vectorize text. This worked well for simple applications but it does have a number of drawbacks. The main ones include:

* There is no way to handle **Out-of-Vocabulary** (OOV) words. If a new word appears in a later document, it will just be dropped. 
* It creates **sparse matrices** which can be inefficient, although we can overcome this by using a dictionary representation. 
* It isn't able to capture similarity between **synonyms**. 
* Word order is lost so words have **no relationship** to each other. For example, "man eats bread" is very different to "bread eats man" but they would have the same representations.

We also came across **static word embeddings**. Whilst this was an improvement to Bag-of-Words because of its ability to capture word meaning, we still had no way to model the sequential nature of language. 

The main challenge with taking word order into account is that sentences can be of **different lengths**. So we need a model that can **automically scale** depending on the sequence length. This is where RNNs come in. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Recurrence</p>

<center>
<img src="https://i.postimg.cc/8cdHgR82/RNN.png" width=650>
</center>
<br>

The **recurrence** in an RNN refers to the **repeating** design of the model. An input word vector ($x_t$) is passed through a **fully connected layer** to generate some activations ($a_t$), which can be used to make predictions ($y_t$) at that time step. But these activations are also passed to the next time step acting as a form of **memory**, which are combined to make the next prediction ($y_{t+1}$). This design is repeated at each time step, so there can be any number of units in the sequence. 

There are **3 weights matrices**, which are used repeatedly used in each cell. $W_x$ are the weights for the input, $W_y$ are the weights for the output and $W_h$ are the weights for the memory. They are combined using the following equations:

<br>
$$
\large
h_t = f(W_x x_t + W_h h_{t-1} + b), \qquad y_t = g(W_y a_t + c)
$$
<br>

where $f$ and $g$ are **activation functions** like ReLU or tanh and $b$ and $c$ are the **biases**. Note that it is common to set $h_0$ to be the zero vector. And not every task requires a prediction at each time step ($y_t$), some only require one at the end of each sentence. 

As an example, if we pass the sentence "She has a cat" through an RNN for part-of-speech tagging; at $t_1$ a prediction is made using "She", at $t_2$ "She has" is used, similarly at $t_3$ "She has a" is used and at $t_4$ the whole sentence is used to make a prediction. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Training RNNs</p>

<center>
<img src="https://i.postimg.cc/pXcZbYpF/BPTT.png" width=600>
</center>
<br>

Training an RNN is **trickier** than a standard feed-forward neural network because of the time aspect. The algorithm used is called **Backpropagation Through Time** (BPTT). The main idea is that when we compute gradients, we need to **reverse all the arrows** in the computation graph and multiply them using the chain rule. 

For example at the final time step, all the input vectors ($x_t$) and all the hidden states ($h_t$) were used to make this prediction so we have to update all of their corresponding weights. As they share weights matrices, we can add each component together. You can find more details in [this article](https://d2l.ai/chapter_recurrent-neural-networks/bptt.html), but luckily all major deep learning frameworks will **implement this for us**. 

# Language Models

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Word prediction</p>

A **Language model** determines the **probability** of a sequence of words. For example, if you consider the incomplete sentence "I'm going to walk the ...", a reasonable guess for the next word would be "dog" as opposed to "ice-cream" or "mountain". We can see that language is **predictable** and some words are more likely to appear than others. 

Mathematically, we can use the **multiplication rule** to represent the probability of a sequence as a product of conditional probabilities:

<br>
$$
\large
\begin{align*}
\mathbb{P}(w_1,...,w_n) &= \prod_{i=1}^{n} \mathbb{P} (w_i | w_1,...,w_{i-1}) \\
& = \mathbb{P}(w_1) \mathbb{P}(w_2|w_1) ... \mathbb{P}(w_n|w_1,...,w_{n-1})
\end{align*}
$$
<br>

This is a **difficult task** because it is impossible to get a training corpus with every combination of words in a sequence. We will see that we can use an RNN to **train** a language model by using a clever trick. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Training an RNN for language modelling</p>

To train an RNN for language modelling, we start by **segmenting** a corpus into chunks, sentences or paragraphs to feed into the model. The longer the sequence length the more **computationally expensive** it is to train but the better the model is at capturing **long range dependencies**. 

We then use a **self-supervised** learning approach. At every time step, we get the RNN to try to **predict the next word** in the sentence in the training corpus. What is great is that we don't need to label the data, as we use the words as the labels themselves. 

The outputs $y_t$ are **probability distributions** over the whole vocabulary. They assign a probability to each word, which represents how likely that word is to appear next given the previous words. A **cross-entropy loss** can be used to measure how good each prediction is. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Text generation</p>

To **generate new text** after training, we modify the RNN to **sample** the output probability distribution at each time step and feed this to be **input** word for following time step, that is $x_{t+1} \sim y_{t}$. 

<br>
<center>
<img src="https://i.postimg.cc/0N0kKYXy/RNN-for-LM.jpg" width=600>
</center>
<br>

So given an input word (this can fixed or randomly sampled), we can generated any amount of text we like. It won't be perfect or grammatically correct by default but it will produce text that has a **similar distribution** to the one it was trained on.

For example, if we use a training corpus comprising of the Lord of the Rings books, then the generated text will look somewhat similar to languge in these books. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Evaluating language models</p>

If we **measure** the output **probabilities** of a language model on a **test set**, we expect better models to produce **higher** probabilities for the correct words. 

A common **metric** to evaluate language models is called **Perplexity** and is defined as:

<br>
$$
\large
\text{Perplexity} = \left(\prod_{i=1}^{T} \mathbb{P}_{LM}(w_i|w_1,...,w_{i-1})\right)^{-1/T} = \sqrt[T]{\prod_{i=1}^{T} \frac{1}{\mathbb{P}_{LM}(w_i|w_1,...,w_{i-1})}}
$$
<br>

Because of the inverse probabilities, the **lower** the perplexity score the **better** and the more natural the language produced. The T-th root is there to **control** for different test lengths.

Note that **humans** have a certain perplexity score too, depending on the corpus, so getting a model's perplexity score too low below that **benchmark** may also produce unnatural text.

# RNN variants

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;"> Drawbracks of simple RNNs</p>

We've now seen the architecture of a **standard** recurrent neural network. Whilst they are big **improvement** in terms of dealing with word order, they do have some shortcomings in their current state. These include:

* Struggling to capture **long range dependencies** because the signal gets **weakens** over time. Simple RNNs have no way of deciding what information is important to 'remember' and what can be 'forgetten'.
* **Vanishing and/or exploding gradients** can occur during training because of the many components affecting each matrix of weights.

There are some ways to **dampen** the affects of these problems, like **gradient clipping**, **batch normalization**, **weight initialization** etc. However, better **cell architectures** have been found, which avoid these problems and produce better results. These include **GRU** and **LSTM**, which we will discuss now. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">GRU</p>

<center>
<img src="https://i.postimg.cc/RV921t3J/gru-cell.png" width=600>
</center>
<br>

GRU stands for **Gated Recurrent Unit** and is a variant of a RNN where the cell/unit has the ability to **forget** information it deems irrelevant. As a result, it tends to do a better job at capturing **long range dependencies** at the cost of increasing the number of **parameters** in the model. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">LSTM</p>

<center>
<img src="https://i.postimg.cc/d1rSrQw9/lstm-cell.jpg" width=500>
</center>
<br>

LSTM stands for **Long Short Term Memory** and is similar to GRU but has more parameters and can manage more complex information transfer. In addition to the normal hidden state that gets passed between units, LSTM has an additional **cell state** that captures **long term memory**. The affect is that the model can make predictions using both short term and long term memory. 

LSTM is more complicated than GRU, so it is able to reach **better performance** on some tasks but again at the cost of increasing the number of **parameters** in the model and therefore taking **longer to train**. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Deep RNNs</p>

<center>
<img src="https://i.postimg.cc/C5LhQQv9/deep-rnn.png" width=250>
</center>
<br>

There are several ways to **combine** recurrent neural networks. One way is to **stack** multiple RNNs on top of each other. We can do this since each RNN takes in a sequence and outputs another sequence of the same length. 

Just like adding another **layer** in a feed-forward neural network, stacking RNNs allows the model to learn more useful **representations** of the data. This can be useful for very **complex** tasks that consist a large number **interactions** in the data, for example machine translation. Again, the thing to keep in mind is that this takes much **longer** to train the model. 

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Bidirectional RNNs</p>

There are some tasks where it makes sense to use not only previous words in the sentence but also **future words** in the sentence. For example, the word "India" in the two sentences "India has a population of 1.4 billion people" and "India got her bag ready to go to school" has **different meaning** based on the words that come **after** it. 

<br>
<center>
<img src="https://i.postimg.cc/XNCkNNg2/bidirectional-rnns.png" width=500>
</center>
<br>

**Bidirectional RNNs** work by adding a **second** RNN that processes the text in **reverse order**. This ensures that the model uses the **whole sentence** each time it makes a prediction. 

Note that the cells in this model can be either **simple RNN, GRU or LSTM units** depending on your preference. It is even possible to **stack** bidirectional RNNs to create **deep bidirectional RNNs**.

# Application

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;"> Part-of-Speech Tagging</p>

We're going to build a **bidirectional LSTM** to perform **Part-of-Speech (PoS) tagging**. We'll use datasets from nltk to train our model

<br>

**Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import requests
import nltk
from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

**Load data**

<br>

We'll use 3 PoS datasets together. Because they all have different tagging schemes, we also download the **universal tagging schema** to ensure they are all tagged **consistently**. Note that this **simplifies** some of the tags, for example example proper noun is converted to noun. 

In [ ]:
# PoS datasets
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

# Use universal tagging system
nltk.download('universal_tagset')

In [ ]:
# Retrieve all PoS-tagged sentences and join them in one list
tagged_sentences = treebank.tagged_sents(tagset='universal') + brown.tagged_sents(tagset='universal') + conll2000.tagged_sents(tagset='universal')

# Example
print("Dataset size:", len(tagged_sentences))
print(tagged_sentences[0])

**Separate features and labels**

In [ ]:
# Initalize
X, y = [], []

# Split sentences into words and tags
for s in tagged_sentences:
    sentence, tags = zip(*s)
    X.append(list(sentence))
    y.append(list(tags))
    
# Example
print(X[0])
print(y[0])

**Train-valid-test split**

In [ ]:
# Create train, validation and test sets
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_tr, y_tr, test_size=0.25, shuffle=True, random_state=0) # 0.25 * 0.8 = 0.2

# Print shapes
print("Train size:", len(X_train))
print("Valid size:", len(X_valid))
print("Test size:", len(X_test))

**Tokenizers**

We need to tokenize both the features and the labels using **two different tokenizers** as they are both sequences.

In [ ]:
# Define tokenizers
word_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tag_tokenizer = keras.preprocessing.text.Tokenizer()

# Fit tokenizers
word_tokenizer.fit_on_texts(X_train)
tag_tokenizer.fit_on_texts(y_train)

# Print vocabulary sizes
print("Word vocabulary size:", len(word_tokenizer.word_index))
print("Tag vocabulary size:", len(tag_tokenizer.word_index))

# Print set of tags
print("\nPossible tags:", tag_tokenizer.word_index)

In [ ]:
# Map tokens to integer ids
X_train_ids = word_tokenizer.texts_to_sequences(X_train)
X_valid_ids = word_tokenizer.texts_to_sequences(X_valid)
X_test_ids = word_tokenizer.texts_to_sequences(X_test)

y_train_ids = tag_tokenizer.texts_to_sequences(y_train)
y_valid_ids = tag_tokenizer.texts_to_sequences(y_valid)
y_test_ids = tag_tokenizer.texts_to_sequences(y_test)

**Padding**

<br>

Padding is a way of making all input sequences the **same length**, by filling shorter sequences with 0's and truncating longer sequences. Although is isn't necessarily required for RNNs, it usually **speeds up training** because it is easier to **batch** training examples together. 

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot([len(X_train_ids[i]) for i in range(len(X_train_ids))], binwidth=1, kde=True)
plt.title('Sentence lengths')
plt.show()

Since most sentences have less than 75 words, we'll **truncate** longer sentences to only include the first 75 words. A more sophisticated approach would be to truncate on a batch-by-batch basis to reduce the number of unnecessary 0's but this will still work well.

In [ ]:
# Maximum length of sequences
max_length = 75

# Pad sequences
X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train_ids, padding='post', maxlen=max_length)
X_valid_pad = keras.preprocessing.sequence.pad_sequences(X_valid_ids, padding='post', maxlen=max_length)
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test_ids, padding='post', maxlen=max_length)

y_train_pad = keras.preprocessing.sequence.pad_sequences(y_train_ids, padding='post', maxlen=max_length)
y_valid_pad = keras.preprocessing.sequence.pad_sequences(y_valid_ids, padding='post', maxlen=max_length)
y_test_pad = keras.preprocessing.sequence.pad_sequences(y_test_ids, padding='post', maxlen=max_length)

# Example
print(X_train_pad[0])

**Build model**

<br>

Our model will consist of an **embedding layer** followed by a **bidirectional LSTM**. 

Note that `mask_zero = True` tells the embedding layer to **ignore padding values** so the model won't make predictions at these time steps. This will make it easier to **properly evaluate** our model as we only care about its performance on non-padded values.

Furthermore, since this is a sequence labelling task we want our model to make a **prediction at each time step**. Setting `return_sequences = True` accomplishes that. 

In [ ]:
# Input/output size (+ 1 for padding token)
num_tokens = len(word_tokenizer.word_index) + 1
num_classes = len(tag_tokenizer.word_index) + 1

# Embedding dimension
embedding_dim = 128

In [ ]:
# Build model
model = keras.Sequential([

    # Map integer ids to trainable word vectors
    layers.Embedding(input_dim = num_tokens, 
                           output_dim = embedding_dim, 
                           input_length = max_length,
                           mask_zero = True),
    
    # Bidirectional LSTM
    layers.Bidirectional(layers.LSTM(128, return_sequences = True, 
                           kernel_initializer = tf.keras.initializers.random_normal(seed=0))),
    
    # Output layer
    layers.Dense(num_classes, activation='softmax', 
                           kernel_initializer=tf.keras.initializers.random_normal(seed=0))
])

# Define optimizer, loss function and accuracy metric
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

# Model summary
model.summary()

**Train model**

<br>

We'll use **early stopping** to prevent overfitting to the train set.

In [ ]:
# Early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

# Train model
history = model.fit(
    X_train_pad, y_train_pad,
    validation_data = (X_valid_pad, y_valid_pad),
    epochs = 20,
    batch_size = 128,
    callbacks = [early_stopping],
    verbose = True
)

The model reaches a reasonably **high accuracy** very **quickly**. After only a few epochs, early stopping kicks in.

**Evaluate on test set**

In [ ]:
# Evaluate model
model.evaluate(X_test_pad, y_test_pad)

**Tag new samples**

<br>

The following function takes a list of sentences as input and returns the models **predicted tags** for each word.

In [ ]:
# Tag new sentences
def tag_new_sentences(sentences):
    # Tokenize new sentences
    X_new_ids = word_tokenizer.texts_to_sequences(sentences)
    
    # Pad sequences
    X_new_pad = keras.preprocessing.sequence.pad_sequences(X_new_ids, padding = 'post', maxlen = max_length)
    
    # Make predictions
    X_new_preds = model.predict(X_new_pad)
    
    # Retrieve most likely tag for each word
    sentence_tags = []
    for i, preds in enumerate(X_new_preds):
        # Extract tags for only non-padded tokens
        tags_seq = [np.argmax(p) for p in preds[:len(X_new_ids[i])]]
        
        # Convert ids back to tokens
        words = [word_tokenizer.index_word[w] for w in X_new_ids[i]]
        tags = [tag_tokenizer.index_word[t] for t in tags_seq]
        
        # zip words and tags together
        sentence_tags.append(list(zip(words, tags)))

    return sentence_tags

In [ ]:
# Example sentences
ex = [
    "Newly discovered green comet comes close to Earth.",
    "Nasa's Mars rover Perseverance completes rock depot.",
]

# Tag sentences
ex_tagged = tag_new_sentences(ex)

# Print results
print(ex_tagged[0])
print('\n',ex_tagged[1])

Nowadays, state-of-the-art PoS taggers use **transformer** models and much **more data** but it's still impressive that we very good results using a recurrent neural network.

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#207d06; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #207d06;">Language Modelling</p>

We're now going to build a language model to **generate text like William Shakespear**. To train this model, we'll use a **text file** that contains the scripts for all of Shakespear's plays. 

In [ ]:
# Load text file
text = open('/kaggle/input/shakespeare-text/text.txt', 'r').read()

# Preview text
print(text[:300])

We're actually going to build a **character level** model because this will be **easier to train** since the target space is significantly smaller compared to a word prediction model.

A downside to predicting characters is the additional challenge of **learning to spell** words correctly. It does make it easier to deal with out of vocabulary words though so there is a trade-off.

**Tokenizer**

<br>

Notice how some **special characters** get included in the vocabulary but all letters has been **lower cased**.

In [ ]:
# Tokenize individual letters
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

# Fit tokenizer
tokenizer.fit_on_texts([text])

# Print vocabulary size
print("Vocabulary size:", len(tokenizer.word_index))

# Print vocabulary
print("\nVocabulary:", tokenizer.word_index)

In [ ]:
# Vectorize characters to intger ids
ids = tokenizer.texts_to_sequences([text])[0]

# Print length of sequence
print("Number of tokens:", len(ids))

**Chunking**

<br>

We now need to **segment** the sequence of ids into **chunks** that will be our training examples. We do this by converting into a **tensorflow dataset** object and applying a **windowing function**. 

We need to **add 1** to the input sequence length because we are going to use the **next character** as the label for the current character. This will ensure we have the same input and target sequence lenghts. The `shift` parameter determines how many characters we shift to the right before creating a new chunk, i.e. it controls how much the chunks **overlap** (if at all). 

In [ ]:
# Tensorflow dataset
slices = tf.data.Dataset.from_tensor_slices(ids)

# Create chunks
input_timesteps = 100
window_size = input_timesteps + 1
windows = slices.window(window_size, shift=50, drop_remainder=True)  # bigger shift reduces number of chunks

# Convert window objects to tensors
dataset = windows.flat_map(lambda window: window.batch(window_size))

# Divide dataset into batches for training
batch_size = 32
batches = dataset.shuffle(10000).batch(batch_size)

# Print example batches
for b in batches.take(2):
    print(b)

**Features and labels**

<br>

Now that we have our chunks, we need to **split** them into training features and labels. Remember that we are using a **self-supervised** approach, so the label is always the character that comes after the current one. We are trying to teach our model to predict the **next character** in the sequence.

In [ ]:
# Split features and labels
Xy_batches = batches.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

# Print first training example
for b in Xy_batches.take(1):
    print("X1 length: ", len(b[0][0].numpy()))
    print("X1: ", b[0][0].numpy())
    print("\ny1 length: ", len(b[1][0].numpy()))
    print("y1: ", b[1][0].numpy())

**Vectorize inputs**

<br>

The last step is map our input integer ids to **vectors**. Since each token is just a character, it doesn't make much sense to use embeddings as there is no sense of character meaning. Instead, we'll just use **one-hot encoding** to be able to pass it to a RNN model. 

In [ ]:
# Number of tokens (+1 of oov tokens)
num_tokens = len(tokenizer.word_index) + 1

# One-hot encode the input sequences
Xy_batches = Xy_batches.map(lambda inputs, labels: (tf.one_hot(tf.cast(inputs, tf.int32), depth=num_tokens), labels))

Prefetching **loads** the data for the **next batch**, while the model is training on the current batch. It can **speed** up the training process.

In [ ]:
# Prefetch data
Xy_batches = Xy_batches.prefetch(tf.data.AUTOTUNE)

**Build model**

<br>

We're now ready to build our model. This time there will be **no imbedding layer** so the input vectors will be fed straight through to **2 stacked LSTM layers** followed by a **dense output layer**. As a result, the **number of parameters** in the model is much **lower** than before.

Note that `recurrent_dropout` is added, which applies dropout **horizontally** across time steps. This will **prevent** the model from **memorizing** long strings of text in the training set. 

In [ ]:
# Build model
model = keras.Sequential([

    # LSTM layers
    layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2),
    layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2),
    
    # Output layer
    layers.Dense(num_tokens, activation='softmax')
])

# Define optimizer and loss function
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')

# Model summary
model.summary()

**Train model**

<br>

This might take some time so we will **save the model** after **every epoch** if it reduces the loss. We'll also include **early stopping** and **learning rate scheduler** callbacks. 

In [ ]:
# Number of epochs
num_epochs = 30

# Early stopping
early_stopping = keras.callbacks.EarlyStopping(
    patience = 3,
    min_delta = 0.0001,
    monitor = 'loss',
    restore_best_weights = True,
)

# Learning rate scheduler
schedule = keras.optimizers.schedules.CosineDecay(initial_learning_rate = 0.002, decay_steps = num_epochs, alpha = 0.0001)
scheduler = keras.callbacks.LearningRateScheduler(schedule, verbose = 0)

# Save best model at every epoch
checkpoint = keras.callbacks.ModelCheckpoint(
    'stacked_lstm',
    monitor = 'loss',
    verbose = 1,
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto',
)

In [ ]:
# Train model
history = model.fit(
    Xy_batches,
    epochs = num_epochs, 
    callbacks = [early_stopping, scheduler, checkpoint]
)

In [ ]:
# Load best model after training completed
model = keras.models.load_model('stacked_lstm')

**Generate new text**

<br>

We can generate new text using the following function. It takes in a portion of text to **kickstart** the model, which then makes predictions up to a **pre-defined maximum** number of characters. 

When sampling from the output distribution, we can adjust it using the **temperature parameter**. The **higher** the temperature the more **random** the predictions, whereas the **lower** the temperature the more **sharpened** the original distribution gets.

In [ ]:
def generate_text(model, tokenizer, seed_text, num_chars = 200, temperature = 1):
    
    text = seed_text
    
    for _ in range(num_chars):
        
        # Use 'input_timesteps' most recent characters as input for the model
        X_new = np.array(tokenizer.texts_to_sequences([text[-input_timesteps:]]))
        
        # One-hot encode input
        X_new = tf.one_hot(X_new, num_tokens)
        
        # Predict probability distribution for next character
        preds = model.predict(X_new)[0, -1:, :]
        
        # Adjust probability distribution using temperature
        preds = tf.math.log(preds) / temperature

        # Sample next character
        next_char_id = tf.random.categorical(preds, num_samples=1)
        next_char = tokenizer.sequences_to_texts(next_char_id.numpy())[0]

        # Add character to running text
        text += next_char

    return text

In [ ]:
%%time

print(generate_text(model, tokenizer, "Romeo took a plane to visit his uncle", num_chars = 200, temperature = 0.2))

In [ ]:
print(generate_text(model, tokenizer, "Juliet was enjoying her new job as a journalist", num_chars = 200, temperature = 0.5))

In [ ]:
print(generate_text(model, tokenizer, "Macbeth put down the knife and walked away", num_chars = 200, temperature = 1))

In [ ]:
print(generate_text(model, tokenizer, "To eat chocolate or not to eat chocolate", num_chars = 200, temperature = 2))

We can see that as the **temperature increases**, the number of **spelling mistakes** increases and the text becomes closer to gibberish. That being said, it is still very impressive how the model learnt how to produce actual **words** in the style of **Shakespeare** by predicting a single character at a time.

# **Conclusion**

In this notebook we looked at how Recurrent Neural Networks and its variants can be used to model text as a **sequence of words**. This allowed to solve a new set of problems, namely **sequence labelling** and **language generation**.

More generally, there are **different ways to set up RNNs** to solve different kinds of problems. 

<br>
<center>
<img src="https://i.postimg.cc/T3XjC3Mw/rnn-setups.jpg" width=600>
</center>
<br>

* **one-to-one** - is equivalent to a feed-forward neural network.
* **one-to-many** - can be used for captioning, decoders or music generation.
* **many-to-one** - is used for classification tasks like sentiment analysis.
* **many-to-many** - used for sequence labelling and language modelling tasks.
* **staggered many-to-many** - (also known as sequence-to-sequence) its applications include translation, summarization and chatbots. 